In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [10]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [11]:
loss_fn = nn.CrossEntropyLoss()

In [12]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
def train_loop(dataloader: DataLoader, model, loss_fn, optimizer: torch.optim.Optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader: DataLoader, model, loss_fn, device):
    model.eval()
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X,y in dataloader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    test_loop(test_dataloader, model, loss_fn, device)
    
print("Done!")

Epoch 1
-------------------------------
loss: 2.299047 [    0/60000]
loss: 2.293570 [ 6400/60000]
loss: 2.278060 [12800/60000]
loss: 2.281901 [19200/60000]
loss: 2.263097 [25600/60000]
loss: 2.234454 [32000/60000]
loss: 2.237485 [38400/60000]
loss: 2.204410 [44800/60000]
loss: 2.211118 [51200/60000]
loss: 2.183177 [57600/60000]
Test Error: 
 Accuracy: 48.9%, Avg loss: 2.177184 

Epoch 2
-------------------------------
loss: 2.182767 [    0/60000]
loss: 2.175952 [ 6400/60000]
loss: 2.121820 [12800/60000]
loss: 2.141500 [19200/60000]
loss: 2.107498 [25600/60000]
loss: 2.039172 [32000/60000]
loss: 2.071383 [38400/60000]
loss: 1.992996 [44800/60000]
loss: 2.006191 [51200/60000]
loss: 1.938577 [57600/60000]
Test Error: 
 Accuracy: 59.5%, Avg loss: 1.931895 

Epoch 3
-------------------------------
loss: 1.958983 [    0/60000]
loss: 1.935873 [ 6400/60000]
loss: 1.818177 [12800/60000]
loss: 1.859192 [19200/60000]
loss: 1.780689 [25600/60000]
loss: 1.702846 [32000/60000]
loss: 1.737035 [38400/